draft augmented version of notebook 9b using Simple Pipeline Executor (SPE)

In [17]:
# copied from existing notebooks 9a, 9b
! echo $IMAGE_DESCRIPTION

w_2022_40


In [18]:
# copied from existing notebook 9a
#import getpass
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas
from astropy.time import Time

import lsst.geom
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler, DatasetType, CollectionType
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [19]:
my_collection_identifier = 'custom_coadd_window1_test1'
my_username = os.environ['USER']
my_collection = 'u/' + my_username + '/' + my_collection_identifier

In [20]:
print(my_collection)

u/ameisner/custom_coadd_window1_test1


In [21]:
ordered_collections_list = [my_collection, '2.2i/runs/DP0.2']
### ordered_collections_list = ['2.2i/runs/DP0.2'] # this was just a test, don't actually want to do this !!

In [22]:
butler = Butler('dp02', collections=ordered_collections_list)

In [23]:
my_tract = 4431
my_patch = 17
my_visits_tupleString = '(919515,924057,924085,924086,929477,930353)' # not used until later
my_dataId = {'band': 'i', 'tract': my_tract, 'patch': my_patch}
my_deepCoadd = butler.get('deepCoadd', my_dataId)

In [24]:
my_deepCoadd_inputs = my_deepCoadd.getInfo().getCoaddInputs()
my_deepCoadd_inputs.visits.asAstropy()

id,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,goodpix,weight,filter
,pix,pix,pix,pix,,,
int64,int32,int32,int32,int32,int32,float64,str32
919515,11900,7900,16099,12099,8982709,3.4656688819793495,i_sim_1.4
924057,11900,7900,16099,12099,16098179,4.384267091685517,i_sim_1.4
924085,11900,7900,16099,12099,831332,4.446833161599578,i_sim_1.4
924086,11900,7900,16099,12099,16136708,4.550420295334223,i_sim_1.4
929477,11900,7900,16099,12099,16280498,4.051326013718346,i_sim_1.4
930353,11900,7900,16099,12099,16076133,3.7685753871220466,i_sim_1.4


In [25]:
len(my_deepCoadd_inputs.visits)

6

In [10]:
# relevant subset of $DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml

#  step3:
#    subset:
#      - makeWarp          ### already done in 9a
#      - assembleCoadd     ### already done in 9a
#      - detection
#      - mergeDetections
#      - deblend
#      - measure
#      - mergeMeasurements
#      - forcedPhotCoadd
#      - transformObjectTable
#      - writeObjectTable
#      - consolidateObjectTable
#      - healSparsePropertyMaps    ### not necessary based on Jira ticket description
#      - selectGoodSeeingVisits    ### not necessary based on Jira ticket description
#      - templateGen               ### not necessary based on Jira ticket description
#    description: |

In [26]:
yaml_file = '$DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml'
steps = 'detection,mergeDetections,deblend,measure,mergeMeasurements,forcedPhotCoadd,transformObjectTable,writeObjectTable,consolidateObjectTable'
my_uri = yaml_file + '#' + steps
print(my_uri)

$DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml#detection,mergeDetections,deblend,measure,mergeMeasurements,forcedPhotCoadd,transformObjectTable,writeObjectTable,consolidateObjectTable


In [27]:
myDetectionPipeline = Pipeline.from_uri(my_uri)

In [13]:
#configs from 9b: 
#  (1) characterizeimage -- 1 psfiteration    ### i think this is actually irrelevant when using the SPE framework?
#  (2) detection -- thresholdValue = 10 (10 sigma)
#  (3) detection -- thresholdType = "stdev"

In [14]:
# myDetectionPipeline.addConfigOverride('CharacterizeImage', 'psfIterations', 1) # this crashes with "LookupError: There are no tasks labeled 'CharacterizeImage' in the pipeline"
#myDetectionPipeline.addConfigOverride('characterizeImage', 'psfIterations', 1) # this crashes with "LookupError: There are no tasks labeled 'characterizeImage' in the pipeline"

In [28]:
myDetectionPipeline.addConfigOverride('detection', 'thresholdValue', 10)
myDetectionPipeline.addConfigOverride('detection', 'thresholdType', "stdev")

In [29]:
queryString = f"tract = {my_tract} AND patch = {my_patch} AND " + \
              f"visit in {my_visits_tupleString} AND skymap = 'DC2'"

print(queryString)

tract = 4431 AND patch = 17 AND visit in (919515,924057,924085,924086,929477,930353) AND skymap = 'DC2'


In [30]:
my_outputCollection = 'u/ameisner/nb9a_spe'

In [31]:
simpleButler = SimplePipelineExecutor.prep_butler('dp02', 
                                                  inputs=ordered_collections_list, 
                                                  output=my_outputCollection)

In [32]:
simpleButler.registry.getCollectionChain(my_outputCollection)

CollectionSearch(('u/ameisner/nb9a_spe/20230508T070540Z', 'u/ameisner/custom_coadd_window1_test1', '2.2i/runs/DP0.2'))

In [33]:
for c in sorted(simpleButler.registry.queryCollections(my_outputCollection+'*')):
    print('Found: ', c)

Found:  u/ameisner/nb9a_spe
Found:  u/ameisner/nb9a_spe/20230508T070540Z


In [34]:
%%time
spe = SimplePipelineExecutor.from_pipeline(myDetectionPipeline, 
                                           where=queryString, 
                                           butler=simpleButler)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/g369a80f31c+b9be13ffc2/python/lsst/pipe/tasks/multiBand.py:261: UserWarning: MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. Use MeasureMergedCoaddSourcesConfig.inputCatalog.
  warnings.warn("MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. "


AttributeError: lsst.pipe.tasks.multiBand.DetectCoaddSourcesConfig has no attribute thresholdValue